## Импорты и проч

In [1]:
# Импортируем необходимые библиотеки и функции
import os
import json
import numpy as np
import pandas as pd
import pysam
import tensorflow as tf

from baskerville import seqnn, gene as bgene
from borzoi_helpers import process_sequence, predict_tracks  # предполагается, что эти функции доступны

# Отключаем лишние предупреждения TensorFlow
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2025-02-14 16:24:25.925089: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 16:24:26.000849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-14 16:24:26.000915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-14 16:24:26.001026: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 16:24:26.028413: I tensorflow/core/platform/cpu_feature_g

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)
!nvcc -V

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2.14.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## Legacy

In [6]:
# %% [code]
# Открываем сборку генома (FASTA)
genome_fasta_path = 'hg38/assembly/ucsc/hg38.fa'
fasta = pysam.Fastafile(genome_fasta_path)

# Получаем список хромосом
chromosomes = [chrom for chrom in fasta.references if chrom.startswith("chr")]
print("Chromosomes:", chromosomes)


Chromosomes: ['chr1', 'chr10', 'chr11', 'chr11_KI270721v1_random', 'chr12', 'chr13', 'chr14', 'chr14_GL000009v2_random', 'chr14_GL000225v1_random', 'chr14_KI270722v1_random', 'chr14_GL000194v1_random', 'chr14_KI270723v1_random', 'chr14_KI270724v1_random', 'chr14_KI270725v1_random', 'chr14_KI270726v1_random', 'chr15', 'chr15_KI270727v1_random', 'chr16', 'chr16_KI270728v1_random', 'chr17', 'chr17_GL000205v2_random', 'chr17_KI270729v1_random', 'chr17_KI270730v1_random', 'chr18', 'chr19', 'chr1_KI270706v1_random', 'chr1_KI270707v1_random', 'chr1_KI270708v1_random', 'chr1_KI270709v1_random', 'chr1_KI270710v1_random', 'chr1_KI270711v1_random', 'chr1_KI270712v1_random', 'chr1_KI270713v1_random', 'chr1_KI270714v1_random', 'chr2', 'chr20', 'chr21', 'chr22', 'chr22_KI270731v1_random', 'chr22_KI270732v1_random', 'chr22_KI270733v1_random', 'chr22_KI270734v1_random', 'chr22_KI270735v1_random', 'chr22_KI270736v1_random', 'chr22_KI270737v1_random', 'chr22_KI270738v1_random', 'chr22_KI270739v1_random'

In [ ]:
# %% [code]
# Создаем структуру для хранения предсказаний
# Для каждого хромосомы будем сохранять список кортежей (start, end, predicted_value)
predicted_tracks = {}

# Определяем, сколько нуклеотидов соответствует одному токену на выходе модели.
# Это значение можно получить из модели, например, так:
token_resolution = models[0].model_strides[0]  # например, 32 нуклеотида на токен

# Входное окно модели (например, 524288 нуклеотидов) делится на три равные части,
# центральная часть будет иметь длину:
central_nucl_length = seq_len // 3

# Число токенов, соответствующее центральной трети:
central_token_length = central_nucl_length // token_resolution

# Импортируем tqdm для отображения прогресс-бара
from tqdm import tqdm

for chrom in chromosomes[:7]: #перебираем только первые, допустим 7, хромосом
    chrom_length = fasta.get_reference_length(chrom)
    chrom_predictions = []  # список кортежей (start, end, value)
    
    # Проходимся по геному с шагом step (где step = seq_len // 3)
    for win_start in tqdm(range(0, chrom_length - seq_len + 1, step),
                          desc=f"Processing {chrom}"):
        win_end = win_start + seq_len
        
        # Извлекаем последовательность из текущего окна в виде one-hot представления
        sequence_one_hot = process_sequence(fasta, chrom, win_start, win_end)
        
        # Получаем предсказание модели для данного окна (например, для wild-type)
        # Если используется ансамбль, можно усреднить предсказания реплик.
        y_pred = predict_tracks(models, sequence_one_hot)
        # Предположим, что y_pred — одномерный массив длины N, где N соответствует числу токенов для всего окна.
        
        # Геномные координаты центральной трети окна:
        center_start = win_start + step       # начало центрального региона
        center_end = win_start + 2 * step       # конец центрального региона
        
        # Вычисляем индексы токенов для центральной части.
        # Если полный выход соответствует окну длины seq_len, то:
        token_start = step // token_resolution  # токен, соответствующий началу центральной трети
        token_end = token_start + central_token_length
        
        # Извлекаем из y_pred токены, соответствующие центральной части.
        central_prediction_tokens = y_pred[token_start:token_end]
        
        # Если центральная часть оказалась пустой, заполняем нулями нужной длины.
        expected_token_length = central_token_length
        if len(central_prediction_tokens) == 0:
            central_prediction_tokens = np.zeros(expected_token_length)
        
        # Преобразуем токены в нуклеотидное разрешение.
        # Повторяем каждое значение token_resolution раз.
        central_prediction_nucl = np.repeat(central_prediction_tokens, token_resolution)
        
        # Ожидаемая длина центрального региона в нуклеотидах:
        expected_length = center_end - center_start
        if len(central_prediction_nucl) > expected_length:
            central_prediction_nucl = central_prediction_nucl[:expected_length]
        elif len(central_prediction_nucl) < expected_length:
            pad_length = expected_length - len(central_prediction_nucl)
            # Используем последнее значение или 0, если массив пуст (хотя по проверке он не пуст)
            pad_value = central_prediction_nucl[-1] if len(central_prediction_nucl) > 0 else 0
            central_prediction_nucl = np.concatenate(
                [central_prediction_nucl, np.full(pad_length, pad_value)]
            )
        
        # Сохраняем для каждой позиции центрального региона предсказанное значение
        for i, pred_val in enumerate(central_prediction_nucl):
            pos_start = center_start + i
            pos_end = pos_start + 1
            chrom_predictions.append((pos_start, pos_end, pred_val))
    
    predicted_tracks[chrom] = chrom_predictions
    print(f"{chrom}: обработано {len(chrom_predictions)} нуклеотидов предсказания")


Processing chr1: 100%|██████████████████████████████████████████████████████████████| 1422/1422 [52:04<00:00,  2.20s/it]


chr1: обработано 248511564 нуклеотидов предсказания


Processing chr10: 100%|███████████████████████████████████████████████████████████████| 763/763 [31:33<00:00,  2.48s/it]


chr10: обработано 133343406 нуклеотидов предсказания


Processing chr11:   4%|██▍                                                           | 31/770 [02:03<1:12:15,  5.87s/it]

In [ ]:
# %% [code]
# Преобразуем предсказания в DataFrame и запишем в файл bedGraph
# Формат bedGraph: chrom, start, end, value
rows = []
for chrom, preds in predicted_tracks.items():
    for start, end, value in preds:
        rows.append((chrom, start, end, value))

df = pd.DataFrame(rows, columns=["chrom", "start", "end", "value"])
output_bedgraph = "predicted_expression.bedGraph"
df.to_csv(output_bedgraph, sep="\t", header=False, index=False)
print("BedGraph сохранен:", output_bedgraph)


In [ ]:
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Файл для сохранения итогового bedGraph
output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

# Обрабатываем только первые 7 хромосом
for chrom in chromosomes[:7]:
    chrom_length = fasta.get_reference_length(chrom)
    chrom_predictions = []  # Здесь будем собирать кортежи: (chrom, start, end, predicted_value)
    
    # Определяем token_resolution
    token_resolution = models[0].model_strides[0]  # Например, 32 нуклеотида на токен
    
    # Проходим по геному с шагом step (где step = seq_len // 3)
    for win_start in tqdm(range(0, chrom_length - seq_len + 1, step), desc=f"Processing {chrom}"):
        win_end = win_start + seq_len
        
        # Извлекаем последовательность из текущего окна (one-hot представление)
        sequence_one_hot = process_sequence(fasta, chrom, win_start, win_end)
        
        # Получаем предсказание модели для текущего окна
        y_pred = predict_tracks(models, sequence_one_hot)
        # Предполагается, что y_pred — одномерный массив длины, равной числу токенов для окна
        
        # Геномные координаты центральной трети окна:
        center_start = win_start + step       # начало центрального региона
        center_end = win_start + 2 * step       # конец центрального региона
        
        # Вычисляем, какие индексы токенов соответствуют центральной трети.
        # token_resolution — число нуклеотидов на токен (например, 32)
        token_start = step // token_resolution  
        central_token_length = (seq_len // 3) // token_resolution
        token_end = token_start + central_token_length
        
        central_prediction_tokens = y_pred[token_start:token_end]
        
        # Если центральная часть пуста, заполняем нулями
        if len(central_prediction_tokens) == 0:
            central_prediction_tokens = np.zeros(central_token_length)
        
        # Апсемплинг: преобразуем токены в нуклеотидное разрешение,
        # повторяя каждое значение token_resolution раз
        central_prediction_nucl = np.repeat(central_prediction_tokens, token_resolution)
        
        # Ожидаемая длина центральной части в нуклеотидах:
        expected_length = center_end - center_start
        if len(central_prediction_nucl) > expected_length:
            central_prediction_nucl = central_prediction_nucl[:expected_length]
        elif len(central_prediction_nucl) < expected_length:
            pad_length = expected_length - len(central_prediction_nucl)
            pad_value = central_prediction_nucl[-1] if len(central_prediction_nucl) > 0 else 0
            central_prediction_nucl = np.concatenate([central_prediction_nucl, 
                                                      np.full(pad_length, pad_value)])
        
        # Сохраняем значение для каждой позиции центрального региона
        for i, pred_val in enumerate(central_prediction_nucl):
            pos_start = center_start + i
            pos_end = pos_start + 1
            chrom_predictions.append((chrom, pos_start, pos_end, pred_val))
    
    # Создаем DataFrame для текущей хромосомы
    df = pd.DataFrame(chrom_predictions, columns=["chrom", "start", "end", "value"])
    
    # Сохраняем результаты на диск в отдельный файл для каждой хромосомы
    chrom_output_file = os.path.join(output_dir, f"{chrom}_predicted_expression.bedGraph")
    
    # Формат bedGraph: chrom, start, end, value
    df.to_csv(chrom_output_file, sep="\t", header=False, index=False, mode="w")
    
    # Очищаем переменные для текущей хромосомы
    del chrom_predictions, df
    gc.collect()  # Принудительно вызываем сборщик мусора
    print(f"{chrom}: сохранено, переходим к следующей хромосоме.")


2025-02-11 10:51:49.569706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902/s]
Processing chr1:   7%|████▌                                                          | 104/1422 [03:46<46:36,  2.12s/it]

In [8]:
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# Папка для сохранения файлов по хромосомам
output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)
print("1")

# Обрабатываем только первые 7 хромосом
for chrom in chromosomes[:7]:
    chrom_length = fasta.get_reference_length(chrom)
    chrom_predictions = []  # Здесь будем собирать кортежи: (chrom, start, end, predicted_value)
    print("2")
    # Определяем token_resolution
    token_resolution = models[0].model_strides[0]  # Например, 32 нуклеотида на токен
    
    # Создаем файл для текущей хромосомы
    chrom_output_file = os.path.join(output_dir, f"{chrom}_predicted_expression.bedGraph")
    
    try:
        # Открываем файл для записи и создаем DataFrame
        with open(chrom_output_file, 'w') as file:

            print("3")


            # Записываем заголовок в файл, если это первый раз
            file.write("chrom\tstart\tend\tvalue\n")
            
            # Проходим по геному с шагом step (где step = seq_len // 3)
            for win_start in tqdm(range(0, chrom_length - seq_len + 1, step), desc=f"Processing {chrom}"):
                win_end = win_start + seq_len
                
                # Извлекаем последовательность из текущего окна (one-hot представление)
                sequence_one_hot = process_sequence(fasta, chrom, win_start, win_end)
                
                # Получаем предсказание модели для текущего окна
                y_pred = predict_tracks(models, sequence_one_hot)
                # Предполагается, что y_pred — одномерный массив длины, равной числу токенов для окна
                
                # Геномные координаты центральной трети окна:
                center_start = win_start + step       # начало центрального региона
                center_end = win_start + 2 * step       # конец центрального региона
                
                # Вычисляем, какие индексы токенов соответствуют центральной трети.
                # token_resolution — число нуклеотидов на токен (например, 32)
                token_start = step // token_resolution  
                central_token_length = (seq_len // 3) // token_resolution
                token_end = token_start + central_token_length
                
                central_prediction_tokens = y_pred[token_start:token_end]
                
                # Если центральная часть пуста, заполняем нулями
                if len(central_prediction_tokens) == 0:
                    central_prediction_tokens = np.zeros(central_token_length)
                
                # Апсемплинг: преобразуем токены в нуклеотидное разрешение,
                # повторяя каждое значение token_resolution раз
                central_prediction_nucl = np.repeat(central_prediction_tokens, token_resolution)
                
                # Ожидаемая длина центральной части в нуклеотидах:
                expected_length = center_end - center_start
                if len(central_prediction_nucl) > expected_length:
                    central_prediction_nucl = central_prediction_nucl[:expected_length]
                elif len(central_prediction_nucl) < expected_length:
                    pad_length = expected_length - len(central_prediction_nucl)
                    pad_value = central_prediction_nucl[-1] if len(central_prediction_nucl) > 0 else 0
                    central_prediction_nucl = np.concatenate([central_prediction_nucl, 
                                                          np.full(pad_length, pad_value)])
                
                # Сохраняем значение для каждой позиции центрального региона
                for i, pred_val in enumerate(central_prediction_nucl):
                    pos_start = center_start + i
                    pos_end = pos_start + 1
                    chrom_predictions.append((chrom, pos_start, pos_end, pred_val))
                
                # Периодическая запись в файл, чтобы не перегружать память
                if len(chrom_predictions) >= 10000:  # Записываем каждые 10000 значений
                    df = pd.DataFrame(chrom_predictions, columns=["chrom", "start", "end", "value"])
                    df.to_csv(file, sep="\t", header=False, index=False, mode="a")
                    chrom_predictions.clear()  # Очищаем накопленные данные
                
            # Записываем оставшиеся данные в файл
            if chrom_predictions:
                df = pd.DataFrame(chrom_predictions, columns=["chrom", "start", "end", "value"])
                df.to_csv(file, sep="\t", header=False, index=False, mode="a")
                
    except Exception as e:
        print(f"Error processing {chrom}: {e}")
    
    # Принудительно очищаем память
    gc.collect()
    print(f"{chrom}: сохранено, переходим к следующей хромосоме.")


1
2
3


2025-02-11 13:27:55.005399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902/s]
Processing chr1: 100%|██████████████████████████████████████████████████████████████| 1422/1422 [56:42<00:00,  2.39s/it]


chr1: сохранено, переходим к следующей хромосоме.
2
3


Processing chr10:   0%|▏                                                                | 2/763 [00:05<33:59,  2.68s/it]


KeyboardInterrupt: 

In [8]:
import os
import numpy as np
import pandas as pd
import gc
import psutil
from tqdm import tqdm

# Папка для сохранения файлов по хромосомам
output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

# Инициализация результатов
chrom_predictions = []  # Список для накопления результатов
print("Start")

# Используем tqdm для прогресс-бара
for index, row in tqdm(filtered_bed_data.iterrows(), total=filtered_bed_data.shape[0], desc="Processing chromosomes"):
    chrom = row['chrom']
    start = row['start']
    end = row['end']
    
    # Извлекаем последовательность из текущего окна (one-hot представление)
    sequence_one_hot = process_sequence(fasta, chrom, start, end)
    
    # Получаем предсказание модели для текущего окна
    y_pred = predict_tracks(models, sequence_one_hot)
    
    # Геномные координаты центральной трети окна
    center_start = start + step       # начало центрального региона
    center_end = start + 2 * step     # конец центрального региона

    # Обработка токенов
    token_resolution = models[0].model_strides[0]  # Например, 32 нуклеотида на токен
    token_start = step // token_resolution
    central_token_length = (seq_len // 3) // token_resolution
    token_end = token_start + central_token_length

    central_prediction_tokens = y_pred[token_start:token_end]

    if len(central_prediction_tokens) == 0:
        central_prediction_tokens = np.zeros(central_token_length)

    central_prediction_nucl = np.repeat(central_prediction_tokens, token_resolution)

    expected_length = center_end - center_start
    if len(central_prediction_nucl) > expected_length:
        central_prediction_nucl = central_prediction_nucl[:expected_length]
    elif len(central_prediction_nucl) < expected_length:
        pad_length = expected_length - len(central_prediction_nucl)
        pad_value = central_prediction_nucl[-1] if len(central_prediction_nucl) > 0 else 0
        central_prediction_nucl = np.concatenate([central_prediction_nucl, 
                                                  np.full(pad_length, pad_value)])

    # Сохраняем для каждой позиции центрального региона предсказанное значение
    for i, pred_val in enumerate(central_prediction_nucl):
        pos_start = center_start + i
        pos_end = pos_start + 1
        chrom_predictions.append((chrom, pos_start, pos_end, pred_val))

    # Проверка использования памяти (50%)
    memory_usage = psutil.virtual_memory().percent
    if memory_usage >= 50:  # если использование памяти больше или равно 50%
        # Записываем данные в файл
        chrom_output_file = os.path.join(output_dir, f"{chrom}_predicted_expression.bedGraph")
        df = pd.DataFrame(chrom_predictions, columns=["chrom", "start", "end", "value"])
        
        # Записываем данные в файл (перезаписываем или дописываем)
        df.to_csv(chrom_output_file, sep="\t", header=False, index=False, mode="a")
        
        # Принудительно очищаем память
        chrom_predictions.clear()
        gc.collect()

    # Периодически сообщаем, что данные сохранены
    if index % 100 == 0:
        print(f"Processed {index} rows. Memory usage: {memory_usage}%")


Start


Processing chromosomes:   0%|                                                                 | 0/13798 [00:00<?, ?it/s]

Preprocessing for chr5 (41626145-41822753) took 0.17 seconds


2025-02-11 16:02:34.363739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902


Inference for chr5 (41626145-41822753) took 12.61 seconds


...

chr11: предсказание сохранено.
Preprocessing for chr6 (164265822-164462430) took 0.16 seconds
Inference for chr6 (164265822-164462430) took 1.81 seconds


Processing chromosomes:   0%|                                                      | 4/13798 [00:23<18:14:01,  4.76s/it]

chr6: предсказание сохранено.
Preprocessing for chr5 (13736747-13933355) took 0.16 seconds


Processing chromosomes:   0%|                                                      | 4/13798 [00:23<22:55:48,  5.98s/it]


KeyboardInterrupt: 

In [ ]:
def infer_chromosome(chrom, region_start, region_end, 
                     center_len, context_len, models, fasta_index):
    """
    Делим [region_start, region_end] на не-пересекающиеся окна center_len,
    для каждого окна добавляем контекст context_len по бокам,
    прогоняем через модель, записываем предикты (только для центральной части).
    """
    # Сформируем список координат центральных окон  [startC, endC) без перекрытий
    # step = center_len, идём от region_start до region_end
    coords = []
    curr_start = region_start
    while curr_start < region_end:
        curr_end = curr_start + center_len
        if curr_end > region_end:
            curr_end = region_end  # можно обрезать последнее окно

        coords.append((curr_start, curr_end))
        curr_start = curr_end  # без перекрытий

    # Пройдёмся по списку окон
    for (center_start, center_end) in coords:
        # 1) учтём контекст
        input_start = center_start - context_len
        input_end   = center_end + context_len

        # чтобы не уйти в отрицательные координаты:
        if input_start < 0:
            input_start = 0

        # и не вылезти за границы хромосомы
        if input_end > chr_sizes[chrom]:
            input_end = chr_sizes[chrom]

        # 2) Получаем one-hot
        seq_one_hot = process_sequence(fasta_index, chrom, input_start, input_end)
        # 3) Предикт
        y_pred = predict_tracks(models, seq_one_hot)  
        y_pred = y_pred.squeeze()  # убираем оси размером 1
        # Убедимся, что у нас двумерный массив [L, C]
        # (если нет, нужно переделать под ваш формат)

        # 4) Нужно извлечь из y_pred только участок, соответствующий центру
        #    т.е. [center_start, center_end) относительно [input_start, input_end).
        offset = center_start - input_start  # индекс в y_pred
        center_len_actual = center_end - center_start
        # предполагаем, что y_pred.shape[0] = (input_end - input_start)
        # Вырежем y_pred[offset : offset + center_len_actual, :]
        center_pred = y_pred[offset : offset + center_len_actual, :]

        # 5) Сохраняем в общий буфер. 
        # Для каждой ткани (bw_name) берём соответствующий канал и формируем intervals
        for bw_name, channel_ix in channels_dict.items():
            channel_values = center_pred[:, channel_ix]
            intervals = []
            for i, val in enumerate(channel_values):
                pos = center_start + i
                intervals.append((pos, pos+1, float(val)))
            chrom_predictions[bw_name].append((chrom, intervals))

        # 6) Проверяем память, если > 50%, «флашим» (сбрасываем) в BigWig
        memory_usage = psutil.virtual_memory().percent
        if memory_usage >= 50:
            print(f"Memory usage {memory_usage}%. Flushing to BigWig...")
            flush_and_clear_buffers()

def flush_and_clear_buffers():
    """Сбрасываем накопленные интервалы во все BigWig файлы, очищаем буфер."""
    for bw_name, bw in bw_writers.items():
        # Выгружаем в bigwig все интервалы, накопленные до сих пор
        for cchrom, cintervals in chrom_predictions[bw_name]:
            bw.addEntries(
                [cchrom]*len(cintervals),
                [iv[0] for iv in cintervals],
                ends=[iv[1] for iv in cintervals],
                values=[iv[2] for iv in cintervals]
            )
        chrom_predictions[bw_name].clear()
    gc.collect()

## Препроцессинг

Код для понимания какой индекс отвечает за какую клеточную линию

In [3]:
import pandas as pd

targets_file = 'targets_gtex.txt'
targets_df = pd.read_csv(targets_file, sep='\t', index_col=0)

# Часто удобно иметь явный столбец индексов внутри датафрейма
targets_df['local_index'] = range(len(targets_df))

# Ткани, которые нас интересуют
tissue_list = [
    "kidney", 
    "liver",
    "adrenal",
    "pancreas",
    "lung"
]

print("Список каналов для нужных тканей:\n")

for tissue in tissue_list:
    # Поищем все строки, где в описании есть это слово/фраза (без учёта регистра)
    matches = targets_df[ targets_df['description'].str.lower().str.contains(tissue.lower()) ]
    
    if len(matches) == 0:
        print(f"Ткань '{tissue}' не найдена в targets_df['description'].")
    else:
        print(f"Ткань '{tissue}':")
        display(matches[['description','local_index']])
        print("-"*50)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Список каналов для нужных тканей:

Ткань 'kidney':


,description,local_index
7560,RNA:kidney,38
7561,RNA:kidney,39
7562,RNA:kidney,40


--------------------------------------------------
Ткань 'liver':


,description,local_index
7563,RNA:liver,41
7564,RNA:liver,42
7565,RNA:liver,43


--------------------------------------------------
Ткань 'adrenal':


...

--------------------------------------------------
Ткань 'pancreas':


,description,local_index
7577,RNA:pancreas,55
7578,RNA:pancreas,56
7579,RNA:pancreas,57


--------------------------------------------------
Ткань 'lung':


,description,local_index
7566,RNA:lung,44
7567,RNA:lung,45
7568,RNA:lung,46


--------------------------------------------------


In [3]:
kidney_ix =  39  
liver_ix =   42  # "left lobe of liver tissue"
adrenal_ix = 4
pancreas_ix =56
lung_ix =    45

# Словарь: название ткани -> индекс канала модели
channels_dict = {
    "ENCFF123KIW_kidney"                 : kidney_ix,
    "ENCFF784MDF_left_lobe_of_liver"     : liver_ix,
    "ENCFF236XOK_adrenal_gland"          : adrenal_ix,
    "ENCFF781TTC_pancreas"               : pancreas_ix,
    "ENCFF242BWW_lung"                   : lung_ix
}

In [17]:
import pandas as pd
import gzip

# Загрузим файл .bed.gz с последовательностями
bed_file_path = '../data/sequences_human.bed.gz'

# Открываем и загружаем данные из .bed файла
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, names=["chrom", "start", "end", "fold"])

# Фильтрация данных по фолду
folds_to_process = ['fold3']  # Можно изменить на нужные фолды
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Сгруппируем по хромосомам
grouped = filtered_bed_data.groupby("chrom")

print(len(filtered_bed_data))
# Посмотрим первые несколько строк
filtered_bed_data.head()

6888


,chrom,start,end,fold
20329,chr5,41626145,41822753,fold3
20330,chr11,40389266,40585874,fold3
20331,chr11,32570759,32767367,fold3
20332,chr6,164265822,164462430,fold3
20333,chr5,13736747,13933355,fold3


## Проверка данных

Убедимся что все записи влезут во входящее окно модели

In [5]:
# Находим максимальную длину последовательности
max_sequence_length = 0

for index, row in filtered_bed_data.iterrows():
    start = row['start']
    end = row['end']
    
    # Вычисляем длину текущей последовательности
    sequence_length = end - start
    
    # Обновляем максимальную длину, если текущая больше
    if sequence_length > max_sequence_length:
        max_sequence_length = sequence_length

print(f"Максимальная длина последовательности: {max_sequence_length} нуклеотидов")


Максимальная длина последовательности: 196608 нуклеотидов


Проверка на пересечения

In [22]:
import pandas as pd
from tqdm import tqdm

overlap_found = False

# Получаем список уникальных хромосом
chromosomes = filtered_bed_data['chrom'].unique()

# Используем tqdm для отображения прогресса
for chrom in tqdm(chromosomes, desc="Checking for overlaps"):
    # Фильтруем по текущей хромосоме
    group = filtered_bed_data[filtered_bed_data['chrom'] == chrom]
    
    # Сортируем записи по старту
    sorted_group = group.sort_values('start')
    
    # Инициализируем предыдущую запись
    prev_row = None
    
    # Проходим по отсортированным записям
    for idx, row in sorted_group.iterrows():
        if prev_row is not None:
            # Проверяем пересечение
            if row['start'] < prev_row['end']:
                print(f"Пересечение на {chrom}: {prev_row[['start', 'end']].to_dict()} и {row[['start', 'end']].to_dict()}")
                overlap_found = True
        prev_row = row

if not overlap_found:
    print("Пересечений не обнаружено в filtered_bed_data.")


Checking for overlaps:   0%|                                                                     | 0/21 [00:00<?, ?it/s]

Пересечение на chr5: {'start': 8770274, 'end': 8966882} и {'start': 8819447, 'end': 9016055}
Пересечение на chr5: {'start': 8819447, 'end': 9016055} и {'start': 8868620, 'end': 9065228}
Пересечение на chr5: {'start': 8868620, 'end': 9065228} и {'start': 8917793, 'end': 9114401}
Пересечение на chr5: {'start': 8917793, 'end': 9114401} и {'start': 8966966, 'end': 9163574}
Пересечение на chr5: {'start': 8966966, 'end': 9163574} и {'start': 9016139, 'end': 9212747}
Пересечение на chr5: {'start': 9016139, 'end': 9212747} и {'start': 9065312, 'end': 9261920}
Пересечение на chr5: {'start': 9065312, 'end': 9261920} и {'start': 9114485, 'end': 9311093}
Пересечение на chr5: {'start': 9114485, 'end': 9311093} и {'start': 9163658, 'end': 9360266}
Пересечение на chr5: {'start': 9163658, 'end': 9360266} и {'start': 9212831, 'end': 9409439}
Пересечение на chr5: {'start': 9212831, 'end': 9409439} и {'start': 9262004, 'end': 9458612}
Пересечение на chr5: {'start': 9262004, 'end': 9458612} и {'start': 93

Checking for overlaps:   5%|██▉                                                          | 1/21 [00:00<00:15,  1.28it/s]

Пересечение на chr5: {'start': 40986896, 'end': 41183504} и {'start': 41036069, 'end': 41232677}
Пересечение на chr5: {'start': 41036069, 'end': 41232677} и {'start': 41085242, 'end': 41281850}
Пересечение на chr5: {'start': 41085242, 'end': 41281850} и {'start': 41134415, 'end': 41331023}
Пересечение на chr5: {'start': 41134415, 'end': 41331023} и {'start': 41183588, 'end': 41380196}
Пересечение на chr5: {'start': 41183588, 'end': 41380196} и {'start': 41232761, 'end': 41429369}
Пересечение на chr5: {'start': 41232761, 'end': 41429369} и {'start': 41281934, 'end': 41478542}
Пересечение на chr5: {'start': 41281934, 'end': 41478542} и {'start': 41331107, 'end': 41527715}
Пересечение на chr5: {'start': 41331107, 'end': 41527715} и {'start': 41380280, 'end': 41576888}
Пересечение на chr5: {'start': 41380280, 'end': 41576888} и {'start': 41429453, 'end': 41626061}
Пересечение на chr5: {'start': 41429453, 'end': 41626061} и {'start': 41478626, 'end': 41675234}
Пересечение на chr5: {'start':

Checking for overlaps:  10%|█████▊                                                       | 2/21 [00:01<00:12,  1.56it/s]

...

Пересечение на chr3: {'start': 138615271, 'end': 138811879} и {'start': 138664444, 'end': 138861052}
Пересечение на chr3: {'start': 138664444, 'end': 138861052} и {'start': 138713617, 'end': 138910225}
Пересечение на chr3: {'start': 138713617, 'end': 138910225} и {'start': 138762790, 'end': 138959398}
Пересечение на chr3: {'start': 138762790, 'end': 138959398} и {'start': 138811963, 'end': 139008571}
Пересечение на chr3: {'start': 138811963, 'end': 139008571} и {'start': 138861136, 'end': 139057744}
Пересечение на chr3: {'start': 138861136, 'end': 139057744} и {'start': 138910309, 'end': 139106917}
Пересечение на chr3: {'start': 138910309, 'end': 139106917} и {'start': 138959482, 'end': 139156090}
Пересечение на chr3: {'start': 138959482, 'end': 139156090} и {'start': 139008655, 'end': 139205263}
Пересечение на chr3: {'start': 139008655, 'end': 139205263} и {'start': 139057828, 'end': 139254436}
Пересечение на chr3: {'start': 139057828, 'end': 139254436} и {'start': 139107001, 'end': 1

Checking for overlaps:  90%|██████████████████████████████████████████████████████▎     | 19/21 [00:13<00:01,  1.92it/s]

Пересечение на chr4: {'start': 23583086, 'end': 23779694} и {'start': 23632259, 'end': 23828867}
Пересечение на chr4: {'start': 23632259, 'end': 23828867} и {'start': 23681432, 'end': 23878040}
Пересечение на chr4: {'start': 23681432, 'end': 23878040} и {'start': 23730605, 'end': 23927213}
Пересечение на chr4: {'start': 23730605, 'end': 23927213} и {'start': 23779778, 'end': 23976386}
Пересечение на chr4: {'start': 23779778, 'end': 23976386} и {'start': 23828951, 'end': 24025559}
Пересечение на chr4: {'start': 23828951, 'end': 24025559} и {'start': 23878124, 'end': 24074732}
Пересечение на chr4: {'start': 23878124, 'end': 24074732} и {'start': 23927297, 'end': 24123905}
Пересечение на chr4: {'start': 23927297, 'end': 24123905} и {'start': 23976470, 'end': 24173078}
Пересечение на chr4: {'start': 23976470, 'end': 24173078} и {'start': 24025643, 'end': 24222251}
Пересечение на chr4: {'start': 24025643, 'end': 24222251} и {'start': 24074816, 'end': 24271424}
Пересечение на chr4: {'start':

Checking for overlaps: 100%|████████████████████████████████████████████████████████████| 21/21 [00:13<00:00,  1.56it/s]

Пересечение на chr10: {'start': 86117709, 'end': 86314317} и {'start': 86166882, 'end': 86363490}
Пересечение на chr10: {'start': 86166882, 'end': 86363490} и {'start': 86216055, 'end': 86412663}
Пересечение на chr10: {'start': 86216055, 'end': 86412663} и {'start': 86265228, 'end': 86461836}
Пересечение на chr10: {'start': 86265228, 'end': 86461836} и {'start': 86314401, 'end': 86511009}
Пересечение на chr10: {'start': 86314401, 'end': 86511009} и {'start': 86363574, 'end': 86560182}
Пересечение на chr10: {'start': 86363574, 'end': 86560182} и {'start': 86412747, 'end': 86609355}
Пересечение на chr10: {'start': 86412747, 'end': 86609355} и {'start': 86461920, 'end': 86658528}
Пересечение на chr10: {'start': 86461920, 'end': 86658528} и {'start': 86511093, 'end': 86707701}
Пересечение на chr10: {'start': 86511093, 'end': 86707701} и {'start': 86560266, 'end': 86756874}
Пересечение на chr10: {'start': 86560266, 'end': 86756874} и {'start': 86609439, 'end': 86806047}
Пересечение на chr10

In [29]:
import pandas as pd
import gzip

# Загрузим данные из bed-файла
bed_file_path = '../data/sequences_human.bed.gz'
with gzip.open(bed_file_path, 'rt') as file:
    bed_data = pd.read_csv(file, sep='\t', header=None, 
                           names=["chrom", "start", "end", "fold"])

# Фильтруем данные по fold3 и fold4
#folds_to_process = ['fold3', 'fold4']
folds_to_process = ['fold3']
filtered_bed_data = bed_data[bed_data['fold'].isin(folds_to_process)]

# Выведем список уникальных хромосом
unique_chroms = filtered_bed_data['chrom'].unique()
print("Уникальные хромосомы в выборке:")
print(unique_chroms)
print(f"Количество уникальных хромосом: {len(unique_chroms)}")

# Для каждой хромосомы находим минимальный start и максимальный end
chrom_stats = filtered_bed_data.groupby("chrom").agg(min_start=('start', 'min'),
                                                      max_end=('end', 'max'))
print("\nМинимальный start и максимальный end по хромосомам:")
print(chrom_stats)

if len(unique_chroms) > 1:
    print("\nВыборка содержит более одной хромосомы.")
else:
    print("\nВыборка содержит только одну хромосому.")


Уникальные хромосомы в выборке:
['chr5' 'chr11' 'chr6' 'chr9' 'chr13' 'chr2' 'chr8' 'chr7' 'chr17' 'chrX'
 'chr18' 'chr12' 'chr15' 'chr19' 'chr1' 'chr16' 'chr20']
Количество уникальных хромосом: 17

Минимальный start и максимальный end по хромосомам:
       min_start    max_end
chrom                      
chr1   143479625  228550247
chr11   25440674   56541083
chr12   32992234   34368994
chr13   40653298  102135774
chr15   20168638   34675550
chr16   33491584   33884884
chr17   81799133   83225066
chr18   13485786   15206757
chr19    3108622    9156817
chr2    68747521  129505661
chr20   30186694   30383302
chr5     8770274  103148800
chr6    99603327  170690035
chr7       10000   74778699
chr8     8158857   55824055
chr9    80588528  138217638
chrX     3230043   58109050

Выборка содержит более одной хромосомы.


# Инференс

In [12]:
import os

# Предполагается, что переменные output_dir, folds_to_process и channels_dict уже определены
output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

# Собираем строку из folds_to_process, например: "fold3" или "fold3_fold4"
folds_str = "_".join(folds_to_process)

# Для каждой ткани создаём по два файла: для прямой (st+) и для обратной (st-) цепи.
chains = ["st+", "st-"]

for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        with open(file_path, "w") as f:
            # Опционально можно записать заголовок трека, если потребуется для визуализации
            f.write(f"track type=bedGraph name=\"{bed_name} {chain}\"\n")

print("Предсозданы файлы:")
for bed_name in channels_dict.keys():
    for chain in chains:
        file_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{chain}_{bed_name}.bedGraph")
        print(file_path)


Предсозданы файлы:
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF123KIW_kidney.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF784MDF_left_lobe_of_liver.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF236XOK_adrenal_gland.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF781TTC_pancreas.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st+_ENCFF242BWW_lung.bedGraph
predicted_expression_by_chromosomes/borzoi_rnaseq_fold3_st-_ENCFF242BWW_lung.bedGraph


In [18]:
center_len = 196_000
context_len = 160_000

# Загрузим FASTA
import pysam
fasta_path = "hg38/assembly/ucsc/hg38.fa"
fasta_index = pysam.Fastafile(fasta_path)

# Получим размеры хромосом
chr_sizes = {chrom: fasta_index.get_reference_length(chrom) for chrom in fasta_index.references}

# Отсортируем хромосомы в "естественном" порядке
def chr_order_key(chrom_name):
    name = chrom_name.replace("chr", "")
    if name.isdigit():
        return (int(name), "")
    elif name == "X":
        return (9999998, "X")
    elif name == "Y":
        return (9999999, "Y")
    elif name in ["M","MT"]:
        return (10000000, "M")
    else:
        return (20000000, name)

chrom_list_sorted = sorted(chr_sizes.keys(), key=chr_order_key)
chrom_list_filtered = [c for c in chrom_list_sorted if c in grouped.groups.keys()]

# Собираем все окна в один список
all_windows = []
for chrom in chrom_list_filtered:
    group = grouped.get_group(chrom)
    region_start = group["start"].min()
    region_end   = group["end"].max()

    # Разбиваем [region_start..region_end] на окна
    curr_start = region_start
    while curr_start < region_end:
        curr_end = curr_start + center_len
        if curr_end > region_end:
            curr_end = region_end
        all_windows.append((chrom, curr_start, curr_end))
        curr_start = curr_end

# Опционально сортируем all_windows глобально (сначала по хромосоме, потом по start)
def window_sort_key(item):
    c, st, en = item
    return (chr_order_key(c), st)

all_windows.sort(key=window_sort_key)

print(f"Total windows to process: {len(all_windows)}")


Total windows to process: 3400


In [36]:
import numpy as np
import time
from tqdm import tqdm
import os

stride = models[0].model_strides[0]

def reverse_complement_onehot(seq_onehot):
    """
    Для массива seq_onehot формы (L, 4),
    где порядок A=0, C=1, G=2, T=3.
    Возвращаем реверс-комплементарную последовательность того же размера (L,4).
    """
    rev = np.flip(seq_onehot, axis=0)
    revcomp = rev[:, [3, 2, 1, 0]]
    return revcomp

def process_window(chrom, center_start, center_end, context_len, models):
    """
    Выполняем:
      1) Добавляем контекст к центральному окну.
      2) Получаем one-hot для + цепи и предсказываем y_pred_plus.
      3) Получаем реверс-комплемент one-hot для − цепи и предсказываем y_pred_minus.
      4) Переводим координаты центрального окна (в нуклеотидах) в индексы выхода (делим на stride).
      5) Возвращаем (center_pred_plus, center_pred_minus) формы (L_out, C).
    """
    # Определяем входной интервал с контекстом
    input_start = max(0, center_start - context_len)
    input_end   = min(chr_sizes[chrom], center_end + context_len)

    # Получаем one-hot для + цепи
    seq_one_hot_plus = process_sequence(fasta_index, chrom, input_start, input_end)
    
    # Предсказываем + цепь
    y_pred_plus = predict_tracks(models, seq_one_hot_plus)
    y_pred_plus = np.squeeze(y_pred_plus)  # ожидается (L_out, C)

    # Получаем one-hot для − цепи (реверс-комплемент)
    seq_one_hot_minus = reverse_complement_onehot(seq_one_hot_plus)
    y_pred_minus = predict_tracks(models, seq_one_hot_minus)
    y_pred_minus = np.squeeze(y_pred_minus)

    # Переводим координаты центрального окна в индексы выхода
    offset = center_start - input_start
    offset_out = offset // stride

    center_len_actual = center_end - center_start
    center_len_actual_out = center_len_actual // stride

    # Проверки на корректность
    if center_len_actual_out <= 0:
        return None, None

    L_plus = y_pred_plus.shape[0]
    if offset_out < 0:
        offset_out = 0
    if offset_out + center_len_actual_out > L_plus:
        center_len_actual_out = L_plus - offset_out
    if center_len_actual_out <= 0:
        return None, None

    center_pred_plus = y_pred_plus[offset_out : offset_out + center_len_actual_out, :]

    L_minus = y_pred_minus.shape[0]
    if offset_out + center_len_actual_out > L_minus:
        center_len_actual_out = L_minus - offset_out
    if center_len_actual_out <= 0:
        return None, None

    center_pred_minus = y_pred_minus[offset_out : offset_out + center_len_actual_out, :]

    return center_pred_plus, center_pred_minus

def run_inference_all_windows(all_windows, models):
    """
    Проходим по всем окнам (единый цикл).
    Для каждого окна делаем:
      - Получаем center_pred_plus и center_pred_minus.
      - Записываем результаты в соответствующие bedGraph-файлы.
    Файлы открываем в режиме 'a' (append), сразу пишем и закрываем, чтобы видеть рост файлов в реальном времени.
    """
    start_time = time.time()
    print(f"Running inference on {len(all_windows)} windows (plus & minus).")

    # Сформируем строку о фолдах
    folds_str = "_".join(folds_to_process)

    for (chrom, center_start, center_end) in tqdm(all_windows, desc="Global windows"):
        center_pred_plus, center_pred_minus = process_window(chrom, center_start, center_end, context_len, models)
        if center_pred_plus is None or center_pred_minus is None:
            # Пропускаем окно, если предсказания отсутствуют
            continue
        
        # Записываем для каждого канала
        for bed_name, channel_ix in channels_dict.items():
            # Если индекс канала выходит за пределы предсказаний, пропускаем
            if channel_ix >= center_pred_plus.shape[1]:
                continue

            # Для + цепи
            channel_values_plus = center_pred_plus[:, channel_ix]
            file_path_plus = os.path.join(
                output_dir,
                f"borzoi_rnaseq_{folds_str}_st+_{bed_name}.bedGraph"
            )
            with open(file_path_plus, "a") as f_plus:
                for i, val in enumerate(channel_values_plus):
                    pos_start = center_start + i  # координаты в нуклеотидах остаются без преобразования
                    pos_end = pos_start + 1
                    if pos_end <= pos_start:
                        continue
                    f_plus.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

            # Для − цепи
            if channel_ix >= center_pred_minus.shape[1]:
                continue
            channel_values_minus = center_pred_minus[:, channel_ix]
            file_path_minus = os.path.join(
                output_dir,
                f"borzoi_rnaseq_{folds_str}_st-_{bed_name}.bedGraph"
            )
            with open(file_path_minus, "a") as f_minus:
                for i, val in enumerate(channel_values_minus):
                    pos_start = center_start + i
                    pos_end = pos_start + 1
                    if pos_end <= pos_start:
                        continue
                    f_minus.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val)}\n")

    elapsed = time.time() - start_time
    print(f"Inference completed in {elapsed:.2f} seconds total.")


In [38]:
print("Start inference...")

# Запускаем
run_inference_all_windows(all_windows, models)

print("All done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.")


Start inference...
Running inference on 3400 windows (plus & minus).


Global windows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3400/3400 [1:55:55<00:00,  2.05s/it]

Inference completed in 6955.56 seconds total.
All done. 10 bedGraph files (5 тканей × 2 цепи) have been saved.


# LEGACY BIGWIG

## Основной цикл инференса

Формируем BigWig-и

In [9]:
import os
import gc
import pysam
import pyBigWig
import numpy as np
import time
import re
from tqdm import tqdm

# Получим размеры хромосом
chr_sizes = {
    chrom: fasta_index.get_reference_length(chrom)
    for chrom in fasta_index.references
}

# Сортируем хромосомы в "естественном" порядке (chr1, chr2, ..., chrX, chrY, etc.)
def chr_order_key(chrom_name):
    name = chrom_name.replace("chr", "")
    if name.isdigit():
        return (int(name), "")
    elif name == "X":
        return (9999998, "X")
    elif name == "Y":
        return (9999999, "Y")
    elif name in ["M","MT"]:
        return (10000000, "M")
    else:
        return (20000000, name)

chrom_list_sorted = sorted(chr_sizes.keys(), key=chr_order_key)

# Фильтруем, чтобы оставить только те хромосомы, что есть в grouped (если нужно)
chrom_list_filtered = [c for c in chrom_list_sorted if c in grouped.groups.keys()]

# Cоздадим заголовок BigWig в ТОМ ЖЕ порядке
chrom_header = [(c, chr_sizes[c]) for c in chrom_list_filtered]


output_dir = "predicted_expression_by_chromosomes/"
os.makedirs(output_dir, exist_ok=True)

# Откроем BigWig файлы в том же порядке
bw_writers = {}
folds_str = "_".join(folds_to_process)  
for bw_name, channel_ix in channels_dict.items():
    bw_path = os.path.join(output_dir, f"borzoi_rnaseq_{folds_str}_{bw_name}.bw")
    bw = pyBigWig.open(bw_path, "w")
    bw.addHeader(chrom_header)  # заголовок
    bw_writers[bw_name] = bw

Функции для инференса

In [12]:
def infer_chromosome(chrom, region_start, region_end, center_len, context_len, models, fasta_index):
    """
    Разбиваем [region_start, region_end] на окна длины center_len (без перекрытий).
    Для каждого окна добавляем context_len слева и справа. 
    Прогоняем через модель -> получаем [L_out, C]. 
    Выделяем центральную часть (center_len) и записываем результаты сразу в BigWig-файлы 
    (по нужным каналам), не буферизуя.
    """
    # Формируем список координат (окна) в порядке возрастания
    coords = []
    curr_start = region_start
    while curr_start < region_end:
        curr_end = curr_start + center_len
        if curr_end > region_end:
            curr_end = region_end
        coords.append((curr_start, curr_end))
        curr_start = curr_end

    start_time = time.time()
    print(f"  Chrom {chrom}: {len(coords)} windows to process...")

    # Обрабатываем окна по одному (прогресс-бар для текущего хрома)
    for (center_start, center_end) in tqdm(coords, desc=f"{chrom}", leave=False):
        # 1) Определяем входной интервал с учётом контекста
        input_start = max(0, center_start - context_len)
        input_end   = min(chr_sizes[chrom], center_end + context_len)

        # 2) Получаем one-hot представление последовательности
        seq_one_hot = process_sequence(fasta_index, chrom, input_start, input_end)

        # 3) Получаем предсказание модели и убираем лишние оси
        y_pred = predict_tracks(models, seq_one_hot)
        y_pred = np.squeeze(y_pred)   # ожидается форма (L_out, C)

        # 4) Определяем смещение и длину центрального окна
        offset = center_start - input_start
        center_len_actual = center_end - center_start

        # Если центральное окно имеет нулевую или отрицательную длину, пропускаем его
        if center_len_actual <= 0:
            continue

        # Если предсказанный массив не содержит достаточного количества позиций, корректируем длину
        available_length = y_pred.shape[0]
        if offset < 0:
            offset = 0
        if offset + center_len_actual > available_length:
            center_len_actual = available_length - offset
            if center_len_actual <= 0:
                continue

        center_pred = y_pred[offset : offset + center_len_actual, :]
        if center_pred.shape[0] == 0:
            continue

        # 5) Для каждого канала (тип клеток) формируем интервалы и сразу записываем в BigWig
        for bw_name, channel_ix in channels_dict.items():
            # Если индекс канала выходит за пределы, пропускаем
            if channel_ix >= center_pred.shape[1]:
                continue

            channel_values = center_pred[:, channel_ix]
            intervals = []
            for i, val in enumerate(channel_values):
                pos_start = center_start + i
                pos_end   = pos_start + 1
                if pos_end <= pos_start:  # защита от некорректных интервалов
                    continue
                intervals.append((pos_start, pos_end, float(val)))
            
            if not intervals:
                continue

            # Дополнительная проверка: все интервалы должны быть корректны
            starts = [iv[0] for iv in intervals]
            ends   = [iv[1] for iv in intervals]
            if any(s >= e for s, e in zip(starts, ends)):
                continue

            bw = bw_writers[bw_name]
            bw.addEntries(
                chromArray = [chrom] * len(intervals),
                startArray = starts,
                endArray   = ends,
                values     = [iv[2] for iv in intervals]
            )

    elapsed = time.time() - start_time
    print(f"  Done {chrom} in {elapsed:.2f} seconds")


Инференс

In [ ]:
print("Start inference...")

center_len = 196_000
context_len = 160_000

for chrom in tqdm(chrom_list_filtered, desc="Chromosomes"):
    # см. group = grouped.get_group(chrom) 
    # Нам нужен min и max (start, end) по этой хромосоме
    group = grouped.get_group(chrom)
    region_start = group["start"].min()
    region_end   = group["end"].max()

    # инференсим сразу в BigWig (без буфера)
    infer_chromosome(chrom, region_start, region_end, 
                     center_len, context_len, models, fasta_index)

for bw_name, bw in bw_writers.items():
    bw.close()

print("Done. All BigWig files have been saved.")

Start inference...


Chromosomes:   0%|                                                                               | 0/21 [00:00<?, ?it/s]

  Chrom chr1: 435 windows to process...



1:  24%|██████████████████                                                         | 105/435 [03:24<10:43,  1.95s/it]